16-数组检索子程序之二
====
indexing-like operations
----

| Indexing-like   operations                  |                                                              |
| ------------------------------------------- | ------------------------------------------------------------ |
| take(a, indices[, axis, out, mode])         | Take   elements from an array along an axis.                 |
| take_along_axis(arr, indices, axis)         | Take   values from the input array by matching 1d index and data slices. |
| choose(a, choices[, out, mode])             | Construct   an array from an index array and a set of arrays to choose from. |
| compress(condition, a[, axis, out])         | Return   selected slices of an array along given axis.       |
| diag(v[, k])                                | Extract   a diagonal or construct a diagonal array.          |
| diagonal(a[, offset, axis1, axis2])         | Return   specified diagonals.                                |
| select(condlist, choicelist[, default])     | Return   an array drawn from elements in choicelist, depending on conditions. |
| lib.stride_tricks.as_strided(x[, shape, …]) | Create   a view into the array with the given shape and strides. |


# numpy.take
`numpy.take(a, indices, axis=None, out=None, mode='raise')`

沿着一个轴从数组中获取元素。

当axis不是None时，此函数与“fancy”索引（使用数组索引数组）的功能相同; 但是，如果您需要沿给定轴的元素，则更容易使用。诸如`np.take（arr，indices，axis = 3）`之类的调用等同于`arr [：，：，：，indices，...]`。

解释没有花哨的索引，这相当于使用了ndindex，它将ii，jj 和 kk 的每一个设置为索引的元组：

```
Ni, Nk = a.shape[:axis], a.shape[axis+1:]
Nj = indices.shape
for ii in ndindex(Ni):
    for jj in ndindex(Nj):
        for kk in ndindex(Nk):
            out[ii + jj + kk] = a[ii + (indices[jj],) + kk]
            ```

## Parameters:	
**a :** array_like (Ni…, M, Nk…), 源数组。

**indices :** array_like (Nj…)，提取的值的索引。也允许用标题来表示索引。

**axis :** int, 可选参数。选择值的轴。默认情况下，使用扁平的输入阵列。

**out :** ndarray, optional (Ni…, Nj…, Nk…)，如果提供了，结果将被放置在这个数组中。它应该是合适的形状和dtype。

**mode :** {‘raise’, ‘wrap’, ‘clip’}, 可选参数。指定越界索引的行为。

‘raise’ – raise an error (default)，默认触发一个错误
‘wrap’ – wrap around，环绕
‘clip’ – clip to the range，剪切到区域

“clip”模式意味着所有太大的索引都被沿着该轴最后一个位置上的元素的索引取代。注意，这将禁用带负数的索引。

## Returns:	
out : ndarray (Ni…, Nj…, Nk…)，返回的数组与 a 有相同的类型。

**注意**

通过消除上面描述中的内部循环，并使用`s_`来构建简单的切片对象，可以对每个`1-d`切片应用花式索引来表示：

```
Ni, Nk = a.shape[:axis], a.shape[axis+1:]
for ii in ndindex(Ni):
    for kk in ndindex(Nj):
        out[ii + s_[...,] + kk] = a[ii + s_[:,] + kk][indices]
        
```
因此，它等同于（但快于）apply_along_axis的以下用法：

`out = np.apply_along_axis(lambda a_1d: a_1d[indices], axis, a)`

## 示例

In [2]:
import numpy as np
>>> a = [4, 3, 5, 7, 6, 8]
>>> indices = [0, 1, 4]
>>> np.take(a, indices)

array([4, 3, 6])

在这个例子中，如果a是ndarray，就可以使用“花式”的索引。

In [3]:
>>> a = np.array(a)
>>> a[indices]

array([4, 3, 6])

如果指标不是一维的，那么输出也会有相同的维度。

In [4]:
>>> np.take(a, [[0, 1], [2, 3]])

array([[4, 3],
       [5, 7]])

# numpy.take_along_axis
`numpy.take_along_axis(arr, indices, axis)`

通过匹配`1d`索引和数据切片从输入数组中获取值。

这个迭代沿着索引和数据数组的指定轴遍历匹配的 1d 切片，并使用前者在后者中查找值。这些切片可以的不同的长度。

函数沿着一个轴返回索引，如 argsort 和 argpartition，为这个函数生成合适的索引。

**New in version 1.15.0.**，在低版本中不可用。

## Parameters:	
**arr:** ndarray (Ni…, M, Nk…)，源数组。

**indices:** ndarray (Ni…, J, Nk…)，

arr数组的每个1d切片的索引。 须与 arr 的大小匹配，但维度 Ni 和  Nj 只需要广播到 arr。

**axis:** int, 沿着这条轴获取 1d 切片。如果axis为None，输入数组被当作第一次被扁平化到 1d，以保持与 sort和 argsort 的一致性。

## Returns:	
out: ndarray (Ni…, J, Nk…)

检索的结果。

**注意：**


这相当于（但更快）以下使用`ndindex`和`s_`，它们将`ii`和`kk`中的每一个设置为索引的元组：

```
Ni, M, Nk = a.shape[:axis], a.shape[axis], a.shape[axis+1:]
J = indices.shape[axis]  # Need not equal M
out = np.empty(Nk + (J,) + Nk)

for ii in ndindex(Ni):
    for kk in ndindex(Nk):
        a_1d       = a      [ii + s_[:,] + kk]
        indices_1d = indices[ii + s_[:,] + kk]
        out_1d     = out    [ii + s_[:,] + kk]
        for j in range(J):
            out_1d[j] = a_1d[indices_1d[j]]
            
```

等效地，消除内部循环，最后两行将是：
`out_1d[:] = a_1d[indices_1d]`

## 示例
对于这样的数组：

In [5]:
>>> a = np.array([[10, 30, 20], [60, 40, 50]])
a

array([[10, 30, 20],
       [60, 40, 50]])

我们可以直接使用 sort 或 argsort 函数进行排序:

In [6]:
>>> np.sort(a, axis=1)

array([[10, 20, 30],
       [40, 50, 60]])

In [7]:
>>> ai = np.argsort(a, axis=1); ai

array([[0, 2, 1],
       [1, 2, 0]], dtype=int64)

** np.take_along_axis(a,ai,axis=1)，该函数在最新的1.15中才有，目前安装的是1.14.3，所以暂不可用。**

# numpy.choose
`numpy.choose(a, choices, out=None, mode='raise')`

从索引数组和一组数组构造一个数组以供选择。

首先，如果感到困惑或不确定，请务必查看示例 - 在一般的完整性上，此函数看起来不如下面的代码描述（在`ndi = numpy.lib.index_tricks`下面）简单：

`np.choose(a,c) == np.array([c[a[I]][I] for I in ndi.ndindex(a.shape)]).`

但这忽略了一些微妙之处。 以下是一个完整的总结：

给定整数的“索引”数组（a）和 n 个数组（choices）的序列，根据需要，首先将 a 和每个 choice 数组广播到共同形状的数组; 调用这些 `Ba` 和 `Bchoices [i]，i = 0，...，n-1`，我们必须为每个 i 设置 `Ba.shape == Bchoices [i] .shape`。 然后，创建一个形状为`Ba.shape`的新数组，如下所示：

- 如果`mode = raise`（默认值），则首先，a（因此Ba）的每个元素必须在`[0，n-1]`范围内; 现在，假设i（在该范围内）是Ba中`（j0，j1，...，jm）`位置的值 - 那么新数组中相同位置的值是`Bchoices [i]`中的值位置;
- 如果`mode = wrap`，则a（因此Ba）中的值可以是任何（带符号）整数; 模运算用于将范围`[0，n-1]`之外的整数映射回该范围; 然后新的数组如上构造;
- 如果`mode = clip`，则a（因此Ba）中的值可以是任何（带符号）整数; 负整数映射到0; 大于n-1的值映射到n-1; 然后如上构造新数组。

## Parameters:	
**a :** int array. 这个数组必须包含`[0、n-1]`中的整数，其中 n 是 choices 的数量，除非 mode=wrap 或mode=clip，在这种情况下，任何整数都是允许的。

**choices :** sequence of arrays，数组的序列。choices数组。 a 和所有 choices 必须可以广播到相同的形状。 如果 choices 本身是一个数组（不推荐），那么它的最外层维度（即，对应于`choices.shape [0]`的那个维度）被认为是定义的“序列”。

**out :** array, 可选参数。如果提供，结果将被插入到这个数组中。它必须有合适的 shape 和 dtype。

**mode :** `{‘raise’ (default), ‘wrap’, ‘clip’}`, 可选参数。

指定如何处理 `[0，n-1]` 之外的索引：
- 'raise'：触发异常
- 'wrap'：value 变为 `value mod n`
- 'clip'：`value<0` 映射到 0，`value > n-1` 映射到 `n-1`

## Returns:	
merged_array : array，合并后的结果。

异常处理：
ValueError：形状不匹配

如果 a 和每个 choice 数组不能全部广播到相同的形状。

**注意**

为了减少误解的可能性，即使名义上支持下列“abuse”，选择既不应该是单个数组，也不应该被认为是单个数组，即最外层的序列式容器应该是列表或元组。

## 示例

In [8]:
>>> choices = [[0, 1, 2, 3], [10, 11, 12, 13],[20, 21, 22, 23], [30, 31, 32, 33]]
choices

[[0, 1, 2, 3], [10, 11, 12, 13], [20, 21, 22, 23], [30, 31, 32, 33]]

In [9]:
>>> np.choose([2, 3, 1, 0], choices)
# the first element of the result will be the first element of the
# third (2+1) "array" in choices, namely, 20; the second element
# will be the second element of the fourth (3+1) choice array, i.e.,
# 31, etc.

array([20, 31, 12,  3])

In [10]:
>>> np.choose([2, 4, 1, 0], choices, mode='clip') 
# 4 goes to 3 (4-1)
>>> # because there are 4 choice arrays

array([20, 31, 12,  3])

In [11]:
>>> np.choose([2, 4, 1, 0], choices, mode='wrap') # 4 goes to (4 mod 4)
# i.e., 0

array([20,  1, 12,  3])

举几个例子说明如何选择广播：

In [12]:
>>> a = [[1, 0, 1], [0, 1, 0], [1, 0, 1]]
>>> choices = [-10, 10]
a

[[1, 0, 1], [0, 1, 0], [1, 0, 1]]

In [13]:
choices

[-10, 10]

In [14]:
>>> np.choose(a, choices)

array([[ 10, -10,  10],
       [-10,  10, -10],
       [ 10, -10,  10]])

In [15]:
>>> # With thanks to Anne Archibald
>>> a = np.array([0, 1]).reshape((2,1,1))
>>> c1 = np.array([1, 2, 3]).reshape((1,3,1))
>>> c2 = np.array([-1, -2, -3, -4, -5]).reshape((1,1,5))
>>> np.choose(a, (c1, c2)) # result is 2x3x5, res[0,:,:]=c1, res[1,:,:]=c2

array([[[ 1,  1,  1,  1,  1],
        [ 2,  2,  2,  2,  2],
        [ 3,  3,  3,  3,  3]],

       [[-1, -2, -3, -4, -5],
        [-1, -2, -3, -4, -5],
        [-1, -2, -3, -4, -5]]])

# numpy.compress
`numpy.compress(condition, a, axis=None, out=None)`

沿着给定的轴返回一个数组的选择切片。

沿给定轴工作时，在条件计算结果为 True 的每个索引的输出中返回沿该轴的切片。在处理1-D阵列时，compress 等同于 extract。

## Parameters:	
**condition :** 1-D array of bools，

用于选择要返回的条目的数组。 如果`len（condition）`小于沿给定轴的 a 的大小，则输出将被截断为 condition 数组的长度。

**a :** array_like。要从中提取一部分的数组。

**axis :** int, 可选参数。沿着该axis进行切片。如果为None(默认)，在数组上扁平化工作。

**out :** ndarray, 可选参数。输出数组，它的类型被保留并且它必须有正确的形状来保持输出。

## Returns:	
compressed_array : ndarray

对于condition为false的axis，沿着该 axis 没有切片的副本。

## 示例

In [16]:
>>> a = np.array([[1, 2], [3, 4], [5, 6]])
>>> a

array([[1, 2],
       [3, 4],
       [5, 6]])

In [17]:
>>> np.compress([0, 1], a, axis=0)

array([[3, 4]])

In [18]:
>>> np.compress([False, True, True], a, axis=0)

array([[3, 4],
       [5, 6]])

In [19]:
>>> np.compress([False, True], a, axis=1)

array([[2],
       [4],
       [6]])

在扁平的数组上工作不会沿着一个轴返回片，而是选择元素。

In [20]:
>>> np.compress([False, True], a)

array([2])

# numpy.diag
`numpy.diag(v, k=0)`

提取对角线或构造对角线数组。

如果您使用此函数提取对角线并希望写入结果数组，请参阅numpy.diagonal的更详细文档; 是否返回副本或视图取决于您正在使用的numpy版本。


## Parameters:	
**v :** array_like，如果 v 是一个 2-D数组，返回第k个对角线的副本。如果 v 是一个1-D数组，返回一个第k个对角线是v的2-D数组。

**k :** int, 可选参数。对角线问题。默认是0。对于在主对角线上的对角线使用 k>0 ，对于在主对角线下的对角线使用 k<0。

## Returns:	
out : ndarray

提取对角线或构造对角线数组。

## 示例

In [21]:
>>> x = np.arange(9).reshape((3,3))
>>> x

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [22]:
>>> np.diag(x)

array([0, 4, 8])

In [23]:
>>> np.diag(x, k=1)

array([1, 5])

In [24]:
>>> np.diag(x, k=-1)

array([3, 7])

In [25]:
>>> np.diag(np.diag(x))

array([[0, 0, 0],
       [0, 4, 0],
       [0, 0, 8]])

# numpy.diagonal
`numpy.diagonal(a, offset=0, axis1=0, axis2=1)`

返回指定的对角线。

如果a是2-D，则返回 a 的具有给定偏移的对角线，即形式为`a [i，i + offset]`的元素的集合。 如果a具有两个以上的维度，则axis1和axis2指定的轴确定一个二维子数组用于返回其对角线。

结果数组的shape可以这样确定：
- 移除axis1和axis2；
- 添加一个索引到右边，等于结果对角线的size。

在1.7之前的NumPy版本中，此函数始终返回一个新的独立数组，其中包含对角线中值的副本。

在NumPy 1.7和1.8中，它继续返回对角线的副本，但取决于此事实已被弃用。写入生成的数组继续像以前一样工作，但会发出FutureWarning。

从NumPy 1.9开始，它返回原始数组的只读视图。 尝试写入结果数组将产生错误。

在将来的某个版本中，它将返回一个读/写视图，写入返回的数组将改变您的原始数组。 返回的数组与输入数组的类型相同。

如果您不写入此函数返回的数组，那么您可以忽略上述所有内容。

如果依赖于当前行为，那么我们建议显式复制返回的数组，即使用np.diagonal（a）.copy（）而不是np.diagonal（a）。 这将适用于NumPy的过去和未来版本。

## Parameters:	
**a :** array_like，从中取出对角线的数组。

**offset :** int, 可选参数。从主对角线偏移对角线。可以是正数或负数。默认为主对角线（0）。

**axis1 :** int, 可选参数。轴用作2-D子阵列的第一轴，应从中取出对角线。默认为第一轴（0）。

**axis2 :** int, 可选参数。轴用作2-D子阵列的第二轴，应从中取出对角线。默认为第二轴（1）。

## Returns:	
array_of_diagonals : ndarray

如果a是2-D，则返回包含对角线且与a类型相同的1-D数组，除非a是矩阵matrix，在这种情况下返回1-D数组而不是（2-D）矩阵，这是为了保持向后兼容性。

如果`a.ndim> 2`，则移除 axis1 和 axis2 指定的尺寸，并在对应于对角线的末端插入新轴。

## Raises:	
ValueError，如果维度小于2.

## 示例

In [26]:
>>> a = np.arange(4).reshape(2,2)
>>> a

array([[0, 1],
       [2, 3]])

In [27]:
>>> a.diagonal()

array([0, 3])

In [28]:
>>> a.diagonal(1)

array([1])

一个3-D的例子：

In [29]:
>>> a = np.arange(8).reshape(2,2,2); a
>>> a.diagonal(0, # Main diagonals of two arrays created by skipping
...            0, # across the outer(left)-most axis last and
...            1) # the "middle" (row) axis first.


array([[0, 6],
       [1, 7]])

刚刚获得的主对角线的子数组; 请注意，每个对应于固定最右侧（列）轴，并且对角线按行“打包”。

In [30]:
>>> a[:,:,0] # main diagonal is [0 6]

array([[0, 2],
       [4, 6]])

In [31]:
>>> a[:,:,1] # main diagonal is [1 7]

array([[1, 3],
       [5, 7]])

# numpy.select
`numpy.select(condlist, choicelist, default=0)`

返回从choicelist中提取元素的组成的数组，具体取决于条件。

## Parameters:	
**condlist :** list of bool ndarrays，布尔数组列表。这个列表

条件列表，用于确定从choicelist中的哪个数组中获取输出元素。当满足多个条件时，使用condlist中遇到的第一个条件。

**choicelist :** list of ndarrays，多维数组列表。

从中获取输出元素的数组列表。 它必须与condlist的长度相同。

**default :** scalar, 可选参数。

当所有条件评估为False时，元素插入输出中。

## Returns:	
output : ndarray

位置 m 的输出是 choicelist 数组的第 m 个元素，其中 condlist 为 True 的相应数组的第m个元素。

## 示例

In [32]:
>>> x = np.arange(10)
>>> condlist = [x<3, x>5]
>>> choicelist = [x, x**2]
>>> np.select(condlist, choicelist)

array([ 0,  1,  2,  0,  0,  0, 36, 49, 64, 81])

# numpy.lib.stride_tricks.as_strided
`numpy.lib.stride_tricks.as_strided(x, shape=None, strides=None, subok=False, writeable=True)`

使用给定的形状和步幅创建数组视图。

**警告**
必须非常小心地使用此功能，请参阅注释。

## Parameters:	
**x :** ndarray，创建一个新数组的数组。

**shape :** sequence of int, 整数序列，可选参数。新数组的形状。默认为 `x.shpae`

**strides :** sequence of int, 可选参数。新数组的步幅stride。默认为x.strides。

**subok :** bool, 布尔，可选参数。如果是 True，子类被保存。

**writeable :** bool, 可选参数。

如果设置为False，则返回的数组将始终是只读的。 否则，如果原始数组是可写的，它将是可写的。 如果可能，建议将其设置为False（参见注释）。

## Returns:	
view : ndarray

## Notes
as_strided在给定精确步幅和形状的情况下创建数组视图。 这意味着它操纵ndarray的内部数据结构，如果操作不正确，数组元素可能指向无效的内存，可能会破坏结果或导致程序崩溃。 建议在计算新步幅时始终使用原始x.strides，以避免依赖连续的内存布局。

此外，使用此函数创建的数组通常包含自重叠内存，因此两个元素是相同的。 这种阵列上的矢量化写操作通常是不可预测的。 它们甚至可能为小型，大型或转置阵列提供不同的结果。 由于必须非常仔细地测试和完成对这些数组的写入，因此您可能希望使用writeable = False来避免意外写入操作。

出于这些原因，建议尽可能避免as_strided。